In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_scheduler,
    default_data_collator,
    SchedulerType
)
import os
import json
from itertools import chain
from datasets import load_dataset

In [2]:
train_file = 'combine-lm.jsonl'
tokenizer = AutoTokenizer.from_pretrained(
    'malaysia-ai/bpe-tokenizer',
)
tokenizer.add_bos_token = False
tokenizer.add_eos_token = False
text_column_name = 'text'

In [ ]:
raw_datasets = load_dataset(
    'json',
    data_files=train_file,
    split='train',
    chunksize=10<<23,
)

In [4]:
train_file

'combine-lm.jsonl'

In [9]:
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

In [10]:
filename = os.path.split(train_file)[1]
column_names = raw_datasets.column_names
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-tokenized',
    num_proc=20,
)

tokenized_datasets

Map (num_proc=20):   0%|          | 0/149263340 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
block_size = 4096

def group_texts(examples):
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    load_from_cache_file=True,
    cache_file_name=f'./{filename}-grouped-{block_size}',
    num_proc=20,
)

Map (num_proc=20):   0%|          | 0/149263340 [00:00<?, ? examples/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
len(lm_datasets) * block_size

16994238464